In [9]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import numpy as np
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

In [34]:
df=pd.read_csv(r"D:\HR_comma_sep.csv")
df

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.10,0.77,6,247,4,0,1,0,sales,low
3,0.92,0.85,5,259,5,0,1,0,sales,low
4,0.89,1.00,5,224,5,0,1,0,sales,low
...,...,...,...,...,...,...,...,...,...,...
14990,0.40,0.57,2,151,3,0,1,0,support,low
14991,0.37,0.48,2,160,3,0,1,0,support,low
14992,0.37,0.53,2,143,3,0,1,0,support,low
14993,0.11,0.96,6,280,4,0,1,0,support,low


In [35]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [24]:
x=np.array([1.2,1.4,1.6,1.8,1.1,1.3])
y=np.array([1,1,0,0,1,1])

In [27]:
init_b,init_w=0.,0.1
eta=0.1

## batched gradient Descent

In [30]:
w,b=init_w,init_b
tol=0.01
for i in range(0,10000):
    y_hat=sigmoid(x*w+b)
    l=log_loss(y,y_hat)
    if l<tol:
        print(f"Algo stopped at iteration {i+1}")
        print(f"weight={w},bias={b}")
        break

    dw=-np.mean((y-y_hat)*x)
    db=-np.mean((y-y_hat))
    w-=eta*dw
    b-=eta*db

print(f'final weight and bias {w},{b}')

final weight and bias -10.90835266722828,16.491477072099823


In [17]:
df=sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [18]:
66df_data=df.iloc[:,[0,2,3,4,5,6,7,8,9,14]]
df_data.head()

,survived,sex,age,sibsp,parch,fare,embarked,class,who,alone
0,0,male,22.0,1,0,7.2500,S,Third,man,False
1,1,female,38.0,1,0,71.2833,C,First,woman,False
2,1,female,26.0,0,0,7.9250,S,Third,woman,True
3,1,female,35.0,1,0,53.1000,S,First,woman,False
4,0,male,35.0,0,0,8.0500,S,Third,man,True


In [20]:
X,y=df_data.drop('survived',axis=1),df_data['survived']
X.shape,y.shape

((891, 9), (891,))

In [27]:
numerical_features=df_data.select_dtypes(include='number').columns[1:]
categorical_nomial_features=['sex','alone']
categorical_ordinal_features=['class','who','embarked']
class_order=['missing','First','Second','Third']
who_order=["missing",'child','woman','man']
embarked_order=['missing','S','C','Q']

ordinal_order=[class_order,who_order,embarked_order]

## Transformation

In [62]:
numerical_pipeline=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='mean')),
    ('scaler',StandardScaler())
])

categorial_nominal_pipeline=Pipeline(steps=[
    ('imputer',SimpleImputer(fill_value='missing',strategy='constant')),
    ('onehot',OneHotEncoder(handle_unknown='ignore',sparse_output='False'))
])

categorial_ordinal_pipeline=Pipeline(steps=[
    ('imputer',SimpleImputer(fill_value='missing',strategy='constant')),
    ('ordinal',OrdinalEncoder(categories=ordinal_order))
])



In [63]:
preprocessor=ColumnTransformer(transformers=[
    ('numerical',numerical_pipeline,numerical_features),
    ('categorical_nominal',categorial_nominal_pipeline,categorical_nomial_features),
    ('categorical_ordinal',categorial_ordinal_pipeline,categorical_ordinal_features)
],remainder='passthrough')

In [64]:
model_pipeline=Pipeline(steps=[
    ('preprocessor',preprocessor.set_output(transform='pandas')),
    ('classifier',LogisticRegression())
])

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [66]:
model_pipeline.fit(X_train,y_train)
y_pred=model_pipeline.predict(X_test)
score=model_pipeline.score(X_test,y_test)

InvalidParameterError: The 'sparse_output' parameter of OneHotEncoder must be an instance of 'bool' or an instance of 'numpy.bool'. Got 'False' instead.

In [51]:
score

0.8208955223880597

In [45]:
preprocessor.fit_transform(X_test)

ValueError: Pandas output does not support sparse data. Set sparse_output=False to output pandas dataframes or disable Pandas output via` ohe.set_output(transform="default").